In [25]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import shutil
from google.cloud import storage
import PIL
from pathlib import Path
import tensorflow as tf


from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay, roc_curve, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

# Getting the current working directory

In [3]:
cwd = os.getcwd()
cwd

'/home/jupyter/Training'

# Copying training and test folder from storage to VM

In [4]:
### copy tar file from storage_4 to VM
#!gsutil -m cp -r gs://storage_4/training_data "$cwd"
#!gsutil -m cp -r gs://storage_4/test "$cwd"

Copying gs://storage_4/training_data/crystal/c_image (14).jpg...
Copying gs://storage_4/training_data/crystal/c_image (284).jpg...
Copying gs://storage_4/training_data/crystal/c_image (286).jpg...
Copying gs://storage_4/training_data/crystal/c_image (289).jpg...               
Copying gs://storage_4/training_data/crystal/c_image (290).jpg...
Copying gs://storage_4/training_data/crystal/c_image (292).jpg...               
Copying gs://storage_4/training_data/crystal/c_image (293).jpg...               
Copying gs://storage_4/training_data/crystal/c_image (295).JPG...
Copying gs://storage_4/training_data/crystal/c_image (296).JPG...
Copying gs://storage_4/training_data/crystal/c_image (297).JPG...               
Copying gs://storage_4/training_data/crystal/c_image (307).JPG...
Copying gs://storage_4/training_data/crystal/c_image (305).JPG...
Copying gs://storage_4/training_data/crystal/c_image (299).JPG...               
Copying gs://storage_4/training_data/crystal/c_image (300).JPG...
Co

In [5]:
image_dir = cwd +"/"+ "training_data/"

# Train / Validation splitting  and  Data augmentation


In [6]:
classes = ['crystal', 'no_crystal']
IMG_SIZE = 224
BATCH_SIZE = 16

# Training set
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                                  rescale=1./255, 
                                  validation_split=0.3,
                                  rotation_range=45, 
                                  horizontal_flip=True,
                                  vertical_flip=True,
                                  brightness_range=(.8,1.2),
                                  fill_mode='nearest'
                                  )

train_generator = train_datagen.flow_from_directory(
                                image_dir,
                                subset='training',
                                class_mode='categorical',
                                shuffle=True, 
                                seed=42, 
                                target_size=(IMG_SIZE, IMG_SIZE),
                                batch_size=BATCH_SIZE
                                )

# Validation set
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                                  rescale=1./255, 
                                  validation_split=0.3
                                  )

val_generator = val_datagen.flow_from_directory(
                                  image_dir,
                                  subset='validation',
                                  class_mode='categorical',
                                  shuffle=True, 
                                  seed=42, 
                                  target_size=(IMG_SIZE, IMG_SIZE),
                                  batch_size=BATCH_SIZE
                                  )


Found 679 images belonging to 2 classes.
Found 290 images belonging to 2 classes.


# Get classes from folder names

In [7]:
train_generator.class_indices

{'crystal': 0, 'no_crystal': 1}

# Shape of training input

In [55]:
for image_batch, labels_batch in train_generator:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(16, 224, 224, 3)
(16, 2)


## Defining the step size

In [8]:
steps_train = round(train_generator.n / BATCH_SIZE)
steps_val = round(val_generator.n / BATCH_SIZE)

# Balancing Classes

In [57]:
class_weights = compute_class_weight(class_weight='balanced',
                                    classes=np.unique(train_generator.classes),
                                    y=train_generator.classes)
class_weights

array([1.86538462, 0.68309859])

In [60]:
# class_weight has to be a dictionary format
class_weight_dict = { i : class_weights[i] for i in range(0, len(class_weights))}
class_weight_dict

{0: 1.8653846153846154, 1: 0.6830985915492958}

In [61]:
# getting number of classes
num_classes = len(class_weights)
num_classes

2

# InceptionV3

In [6]:
# define input tensor for model = target shape of image and the number of channels. 
# If the images are colored, then put 3
input_t = tf.keras.Input(shape = (224,224,3))

# load a new instance of model
InceptionV3_model = tf.keras.applications.InceptionV3(input_tensor = input_t,
                                              include_top=False, # do not include ImageNet classifier at the top
                                              weights= "imagenet")

ResourceExhaustedError: OOM when allocating tensor with shape[3,3,80,192] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomUniform]

In [62]:
# to get an overview of the model architecture
InceptionV3_model.summary()

NameError: name 'InceptionV3_model' is not defined

# Transfer Learning of InceptionV3


In [ ]:
# Making the model trainable in general except for the batch normalization layers which should never be trained
InceptionV3_model.trainable = True
for layer in InceptionV3_model.layers:
    if('batch_normalization_' in layer.name):
    layer.trainable = False

In [ ]:
# To check which layers were opened
for layer in InceptionV3_model.layers:
    print(layer.name, '->', layer.trainable)

In [ ]:
# Fine-tune from this layer onward. This means most layers remain closed and will not be trained
fine_tune_at = 287

# Freeze all the layers before the `fine_tune_at` layer
for layer in InceptionV3_model.layers[:fine_tune_at]:
    layer.trainable =  False

In [ ]:
# To check which layers were opened after you opened some
for layer in InceptionV3_model.layers:
    print(layer.name, '->', layer.trainable)

In [ ]:
# points to last layer which is necessary for the new model below
last_conv_layer = InceptionV3_model.get_layer('mixed10')

# Setting up New_Model

In [ ]:
num_classes = len(classes)

## Inception 
conv_model = Model(inputs=InceptionV3_model.input,
                   outputs=last_conv_layer.output)

## Start a new Keras Sequential model 
new_model = Sequential()

# Add the convolutional part of the model from above 
new_model.add(conv_model)

## Add a global average pooling layer before the final dense layer
new_model.add(tf.keras.layers.GlobalAveragePooling2D())

## adding dropout layer to further prevent overfitting
new_model.add(tf.keras.layers.Dropout(0.2))

## Add a final dense (=fully-connected) layer.
## This is for combining features recognized in the image and calculating predictions 
## note that the number of nodes is the sameas the number of classes 
new_model.add(tf.keras.layers.Dense(num_classes, activation='softmax')) 

# Optimizer

In [ ]:
# defining an optimizer for compilation
optimizer = Adam(lr=1e-3)
optimizer.lr.numpy()

## Compile the model

In [ ]:
# Categorical Crossentropy
new_model.compile(optimizer= optimizer,
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

## Model summary

In [ ]:
# complete model layers
for layer in new_model.layers:
    print(layer.name, '->', layer.trainable)

In [ ]:
# complete model summary
new_model.summary()

## Train the model

Tthe more epochs the longer the training takes but also, it will perform better. 
For the original pre-trained model which you can load (in prediction notebook), 80 epochs were used.
If you just want to have a quick glance at the results, put 5. 

In [ ]:
epochs = 80
history = new_model.fit(
  train_generator,
  steps_per_epoch=steps_train,
  validation_data=val_generator,
  validation_steps = steps_val,
  # callbacks = [tboard_callback],
  class_weight = class_weight_dict,
  epochs=epochs
)

In [ ]:
# This is only needed if further training based on the previous history is required.
# Instead of running the entire training from the beginning, you can start where you left of.
# eg. you trained for 30 epochs above and want to train the model for another 20 epochs, just put 20 here
# and the model will start training from epoch 31 on. 

new_epochs = 20
epochs += new_epochs

# training including previous training
history_2 = new_model.fit(
  train_generator,
  initial_epoch=history.epoch[-1]+1,
  epochs = epochs,
  steps_per_epoch=steps_train,
  validation_data=val_generator,
  validation_steps = steps_val,
  class_weight = class_weight_dict
)

# Save Model
If the model from above is any good it should be saved to the storage with bucket name (for Moritz to define). For now, let's call it "model_storage"

In [ ]:
# new_model.save(cwd+"/InceptionV3_XXX")

In [ ]:
# !gsutil -m cp -r "InceptionV3_XXX" gs://model_storage/

## Visualize training results  

In [ ]:
acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Predict on test set

In [19]:
classes = ['crystal', 'no_crystal']
IMG_SIZE = 224
folder_path = cwd + "/test/"

predictions = {}

for filename in os.listdir(folder_path):
    if filename.endswith(".jpg") or filename.endswith(".JPG"):
        img = keras.preprocessing.image.load_img(folder_path+filename,
                                               target_size=(IMG_SIZE, IMG_SIZE))
        img_array = keras.preprocessing.image.img_to_array(img)/255
        img_array = tf.expand_dims(img_array, 0)
        pred = new_model.predict(img_array)
        predictions[filename] = (classes[np.argmax(pred)],
                               ("confidance of {:.2f}%".format(100 * np.max(pred))))
        continue
    else:
        continue

predictions

NameError: name 'new_model' is not defined

# Evaluate

## Confusion Matrix 

In [20]:
# create data frame with true labels
df_json = pd.read_json(cwd +"/image_labels.json", orient="columns")
#df_json = df_json.set_index("index")
#df_json.head(2)

In [ ]:
#create data frame with predicted labels
df_pred = pd.DataFrame.from_dict(predictions, orient="index").reset_index()                    
df_pred.columns = ["image", "predictions","confidence"]
df_pred = df_pred.set_index("image")
# df_pred.head(2)

In [ ]:
# concatenate the two data frames
cm_df = pd.concat([df_pred, df_json], axis=1, join='inner')
# cm_df

In [ ]:
conf_mat = confusion_matrix(cm_df.loc[:,"y_true"], cm_df.loc[:,"predictions"])
cm_display = ConfusionMatrixDisplay(conf_mat, display_labels=['crystal', 'no_crystal']).plot()

## Classification Report

In [ ]:
cr = classification_report(cm_df.loc[:,"y_true"], 
                           cm_df.loc[:,"predictions"], 
                           labels = ['crystal', 'no_crystal'], 
                           digits=2, 
                           zero_division='warn')
print(cr)

## ROC curve

In [ ]:
cm_df.replace(to_replace="no_crystal", value=1, inplace=True)
cm_df.replace(to_replace="crystal", value=0, inplace=True)

In [ ]:
def plot_roc(y_test, proba_preds):

    # create linear line
    base_probs = [0 for _ in range(len(y_test))]

    base_auc = roc_auc_score(y_test, base_probs)
    lr_auc = roc_auc_score(y_test, proba_preds)

    # summarize scores
    print('Logistic: ROC AUC=%.3f' % (lr_auc))

    # calculate roc curves
    ns_fpr, ns_tpr, _ = roc_curve(y_test, base_probs)
    lr_fpr, lr_tpr, _ = roc_curve(y_test, proba_preds)

    # plot the roc curve for the model
    plt.plot(ns_fpr, ns_tpr, linestyle='--', label='Base')
    plt.plot(lr_fpr, lr_tpr, marker='.', label='InceptionV3')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend()
    plt.show()

In [ ]:
plot_roc(cm_df.loc[:,"y_true"], 
         cm_df.loc[:,"predictions"])

# Loading a pre-trained model
In case you only want to predict on new data, jump to this part of the notebook

In [21]:
#this is the path to the pre-trained model in case you don't want to train on a new model
path_pred = Path("/home/jupyter/Prediction/Inception_Pre-trained_Model")

In [23]:
# make sure to load the correct model in case you reset the kernel
new_model = tf.keras.models.load_model(path_pred)

ResourceExhaustedError: OOM when allocating tensor with shape[3,3,288,384] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomUniform]

In [ ]:
IMG_SIZE = 224
folder_path = cwd + "/test/"

predictions = {}

for filename in os.listdir(folder_path):
    if filename.endswith(".jpg") or filename.endswith(".JPG"):
        img = keras.preprocessing.image.load_img(folder_path+filename,
                                               target_size=(IMG_SIZE, IMG_SIZE))
        img_array = keras.preprocessing.image.img_to_array(img)/255
        img_array = tf.expand_dims(img_array, 0)
        pred = new_model.predict(img_array)
        predictions[filename] = (classes[np.argmax(pred)],
                               ("confidance of {:.2f}%".format(100 * np.max(pred))))
        continue
    else:
        continue

predictions

# Delete training and test folder which were copied to the VM

In [ ]:
# specify the directories to delete within the location
folder_to_del = cwd + "/training_data/"

# removing directory 
shutil.rmtree(folder_to_del, ignore_errors = False)

In [26]:
# specify the directories to delete within the location
folder_to_del = cwd + "/test"

# removing directory 
shutil.rmtree(folder_to_del, ignore_errors = False)